In [ ]:
!ls /usr/lib/jvm

java-1.11.0-openjdk-amd64  java-11-openjdk-amd64


In [ ]:
# !apt-get update
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# !wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
#  # Unzip file
# !tar -xvzf spark-3.5.0-bin-hadoop3.tgz

In [1]:
!pip install -q findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=ea55568a35ff6aa884f8e4f0a1997fb868763a7b45d4e8c41c03f36157bc473a
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
!pip install python-levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 10.6 MB/s eta 0:00:00


In [ ]:
#import findspark
#findspark.init()

In [3]:
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType
from pyspark.sql.functions import split
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col, concat_ws
from pyspark.sql.functions import regexp_replace, regexp_extract, collect_list, explode, udf
from pyspark.sql.types import DoubleType
from Levenshtein import distance, ratio

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!ls /content/gdrive/MyDrive/'Colab Notebooks'/Dia_Exercise/

acm_1995_2004.csv		     dblp_1995_2004.csv     dia_assignments_v2.ipynb
acm_1995_2004_specialCharsTitle.csv  dblp.txt
acm.txt				     dia_assignments.ipynb


In [ ]:
!ls /content/gdrive/MyDrive/TestMamun.txt

/content/gdrive/MyDrive/TestMamun.txt


In [ ]:
file_path_dblp = "/content/gdrive/MyDrive/Colab Notebooks/Dia_Exercise/dblp.txt"
file_path_acm = "/content/gdrive/MyDrive/Colab Notebooks/Dia_Exercise/acm.txt"

In [5]:
file_path = "/content/gdrive/MyDrive/TestMamun_2.txt"

In [ ]:
file_path = file_path_dblp

In [6]:
spark = SparkSession.builder.appName("RDDPrintExample").getOrCreate()

# Define the custom delimiter
delimiter = "\n\n"

# Create an RDD using newAPIHadoopFile with TextInputFormat
rdd = spark.sparkContext.newAPIHadoopFile(
    file_path,
    "org.apache.hadoop.mapreduce.lib.input.TextInputFormat",
    "org.apache.hadoop.io.LongWritable",
    "org.apache.hadoop.io.Text",
    conf={"textinputformat.record.delimiter": delimiter},
)


In [7]:
# Define the schema
pub_schema = StructType([
    StructField("title", StringType(), True),
    StructField("authors", StringType(), True),
    StructField("year", StringType(), True),
    StructField("journal", StringType(), True),
    StructField("index", StringType(), True),
])

In [ ]:
data = rdd.filter(lambda x: not x[1].startswith('#%')).map(lambda x: tuple(
    (
        next((field[2:] for field in x[1].splitlines() if field.startswith('#*')), None),  # Paper Title
        next((field[2:] for field in x[1].splitlines() if field.startswith('#@')), None),  # Authors
        next((field[2:] for field in x[1].splitlines() if field.startswith('#t')), None),  # Year
        next((field[2:] for field in x[1].splitlines() if field.startswith('#c')), None),  # Publication Venue
        next((field[6:] for field in x[1].splitlines() if field.startswith('#index')), None)  # Index ID
    )
))

In [ ]:
for el in data.take(5):
    print(el)

('INFORMS Journal on Computing', None, '2014', 'INFORMS Journal on Computing', '558ac6e0612c41e6b9d39eed')
('Pushout-complements and basic concepts of grammars in toposes', 'Yasuo Kawahara', '1990', 'Theoretical Computer Science', '5390879920f70186a0d422b8')
('Effective constructors the formal series of trees (French)', 'Symeon Bozapalidis', '1990', 'Theoretical Computer Science', '5390879920f70186a0d422b6')
('The DataPaper: living in the virtual world', 'Mark Green, Chris Shaw', '1990', 'Graphics Interface 1990', '555aa9a345ce207198fe0ae8')
('Using program slicing in software maintenance', 'Keith Brian Gallagher', '1990', 'Using program slicing in software maintenance', '5390879920f70186a0d422ab')


In [ ]:
# # Apply the schema to the RDD 'a'
df = spark.createDataFrame(data, schema=pub_schema)
df.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------+-------------------------------+----+----------------------------------------------------------------------------------------------------------------------------+------------------------+
|title                                                                                                                       |authors                        |year|journal                                                                                                                     |index                   |
+----------------------------------------------------------------------------------------------------------------------------+-------------------------------+----+----------------------------------------------------------------------------------------------------------------------------+------------------------+
|INFORMS Journal on Computing                             

In [ ]:
def create_dataframe_from_file(file_path, pub_schema):
    # Define the custom delimiter
    delimiter = "\n\n"

    # Create an RDD using newAPIHadoopFile with TextInputFormat
    rdd = spark.sparkContext.newAPIHadoopFile(
        file_path,
        "org.apache.hadoop.mapreduce.lib.input.TextInputFormat",
        "org.apache.hadoop.io.LongWritable",
        "org.apache.hadoop.io.Text",
        conf={"textinputformat.record.delimiter": delimiter},
    )

    # Filter and map the RDD to extract relevant fields
    data_rdd = rdd.filter(lambda x: not x[1].startswith('#%')).map(lambda x: tuple(
        (
            next((field[2:] for field in x[1].splitlines() if field.startswith('#*')), None),  # Paper Title
            next((field[2:] for field in x[1].splitlines() if field.startswith('#@')), None),  # Authors
            next((field[2:] for field in x[1].splitlines() if field.startswith('#t')), None),  # Year
            next((field[2:] for field in x[1].splitlines() if field.startswith('#c')), None),  # Publication Venue
            next((field[6:] for field in x[1].splitlines() if field.startswith('#index')), None)  # Index ID
        )
    ))

    # Create DataFrame using the defined schema
    df = spark.createDataFrame(data_rdd, schema=pub_schema)

    return df

df = create_dataframe_from_file(file_path, pub_schema)
df.show(truncate=False)


+--------------------------------------------------------------------------------+------------------------------------------------------------+----+------------------------------------------+------------------------+
|title                                                                           |authors                                                     |year|journal                                   |index                   |
+--------------------------------------------------------------------------------+------------------------------------------------------------+----+------------------------------------------+------------------------+
|Protein Kinase C, Models of.                                                    |Kim L. Blackwell                                            |2014|Encyclopedia of Computational Neuroscience|55503da645ce0a409eb273e8|
|Stimulus Reconstruction from Cortical Responses.                                |Nima Mesgarani                                    

In [ ]:
def filter_and_clean_df(df):
    # Filter publications between 1995 and 2004 in VLDB and SIGMOD venues
    filtered_df = df.filter(
        (col("year").cast("int").between(1995, 2004)) &
        (col("journal").rlike("(?i)SIGMOD|VLDB"))
    )

    # Clean the journal column
    cleaned_df = filtered_df.withColumn("journal",
                                        regexp_replace(
                                            regexp_replace("journal", "(?i).*\\bVLDB\\b.*", "VLDB"),
                                            "(?i).*\\bSIGMOD\\b.*", "SIGMOD"))

    return cleaned_df

In [ ]:
def remove_special_chars(df):
  pattern = r'[^\w,\s]'
  return df.withColumn("title", regexp_replace("title", pattern,'')).withColumn("authors", regexp_replace("authors", pattern,''))

In [8]:
def write_to_csv(df, path):
  df.repartition(1).write.format('csv').option("header", "true").option("quote", "\"").option("escape", "\"").save(path, mode='overwrite')

In [9]:
def levenshtein_ratio(s1, s2):
    return ratio(s1, s2)

# Register the UDF to calculate Levenshtein ratio
levenshtein_ratio_udf = udf(levenshtein_ratio, DoubleType())

def find_duplicates_for_ground_truth(df1, df2, threshold=0.5):
    # Select only the "title" and "authors" columns from df1 and alias them
    df1_subset = df1.select(col("title").alias("title_1"), col("authors").alias("authors_1"))

    # Select only the "title" and "authors" columns from df2 and alias them
    df2_subset = df2.select(col("title").alias("title_2"), col("authors").alias("authors_2"))

    # Join the subsets of df1 and df2 containing "title" and "authors" columns
    joined_df = df1_subset.crossJoin(df2_subset)

    # Calculate the similarity score using Levenshtein ratio on "title" and "authors"
    similarity_df = joined_df.withColumn("Title_Similarity", levenshtein_ratio_udf(joined_df["title_1"], joined_df["title_2"])) \
                             .withColumn("Authors_Similarity", levenshtein_ratio_udf(joined_df["authors_1"], joined_df["authors_2"]))

    # Filter the DataFrame to keep only the pairs with similarity scores greater than or equal to the specified threshold
    duplicates_df = similarity_df.filter((similarity_df["Title_Similarity"] >= threshold) & (similarity_df["Authors_Similarity"] >= threshold))

    # Select and rename the relevant columns
    duplicates_df = duplicates_df.select(duplicates_df["title_1"].alias("Title1"),
                                         duplicates_df["title_2"].alias("Title2"),
                                         duplicates_df["authors_1"].alias("Authors1"),
                                         duplicates_df["authors_2"].alias("Authors2"),
                                         duplicates_df["Title_Similarity"],
                                         duplicates_df["Authors_Similarity"])

    return duplicates_df


In [ ]:
df_dblp = create_dataframe_from_file(file_path_dblp, pub_schema)
#df_acm = create_dataframe_from_file(file_path_acm, pub_schema)

In [ ]:
df_dblp_clean = remove_special_chars(filter_and_clean_df(df_dblp))
#df_acm_clean = remove_special_chars(filter_and_clean_df(df_acm))

In [ ]:
df_dblp_clean.show()

+--------------------+--------------------+----+-------+--------------------+
|               title|             authors|year|journal|               index|
+--------------------+--------------------+----+-------+--------------------+
|An initial study ...|      Amol Deshpande|2004| SIGMOD|53e9a515b7602d970...|
|Engineering Feder...|Stefan Conrad, Wi...|1999| SIGMOD|53e9b275b7602d970...|
|Information Findi...|Tak W Yan, Hector...|1995| SIGMOD|53e9a5beb7602d970...|
|       Editors Notes|      Jennifer Widom|1995| SIGMOD|53e99800b7602d970...|
|Report on the 5th...|HansJoachim Lenz,...|2003| SIGMOD|53e9a718b7602d970...|
|       Editors Notes|            Ling Liu|2002| SIGMOD|53e99800b7602d970...|
|Report from the N...|Amit P Sheth, Dim...|1996| SIGMOD|53e99e6ab7602d970...|
|TODS Perceptions ...| Richard T Snodgrass|2002| SIGMOD|53e99ae6b7602d970...|
|SQLMED  A Status ...|Jim Melton, JanEi...|2002| SIGMOD|53e9b4d4b7602d970...|
|2003 SIGMOD Innov...| Donald D Chamberlin|2003| SIGMOD|53e9b4a5

In [10]:
path_dblp_csv = "/content/gdrive/MyDrive/Colab Notebooks/Dia_Exercise/dblp_1995_2004.csv"
#write_to_csv(df_dblp_clean, path_dblp_csv)

In [ ]:
df_acm_clean.show()

+--------------------+--------------------+----+-------+--------------------+
|               title|             authors|year|journal|               index|
+--------------------+--------------------+----+-------+--------------------+
|The next database...|            Jim Gray|2004| SIGMOD|5390972920f70186a...|
|The role of crypt...|         Ueli Maurer|2004| SIGMOD|5390972920f70186a...|
|Tree logical clas...|Stelios Paparizos...|2004| SIGMOD|5390972920f70186a...|
|Adaptive stream r...|Ankur Jain, Edwar...|2004| SIGMOD|5390972920f70186a...|
|Holistic UDAFs at...|Graham Cormode, T...|2004| SIGMOD|5390972920f70186a...|
|Online eventdrive...|Huanmei Wu, Betty...|2004| SIGMOD|5390972920f70186a...|
|Using the structu...|Kristina Lerman, ...|2004| SIGMOD|5390972920f70186a...|
|FleXPath flexible...|Sihem AmerYahia, ...|2004| SIGMOD|5390972920f70186a...|
|An interactive cl...|Wensheng Wu, Clem...|2004| SIGMOD|5390972920f70186a...|
|Lazy query evalua...|Serge Abiteboul, ...|2004| SIGMOD|53909729

In [ ]:
duplicates_find_naive = find_duplicates_for_ground_truth(df_acm_clean, df_dblp_clean)

In [11]:
csv_path_duplicates = "/content/gdrive/MyDrive/Colab Notebooks/Dia_Exercise/duplicates_dblp.csv"

In [ ]:
write_to_csv(duplicates_find_naive, csv_path_duplicates)

In [14]:
def read_csv_with_schema(spark, file_path, schema):
    """
    Read records from a CSV file using a specified schema.

    Args:
    - spark: SparkSession object
    - file_path: path to the CSV file
    - schema: schema to be applied to the DataFrame

    Returns:
    - DataFrame containing the records from the CSV file with the specified schema
    """
    # Read CSV file with schema
    df = spark.read.csv(file_path, schema=schema, header=True)

    return df

# file_path_acm = "/kaggle/input/pubdata/acm_1995_2004.csv"
# file_path_dblp = "/kaggle/input/pubdata/dblp_1995_2004.csv"

# Read CSV file with schema
df_acm = read_csv_with_schema(spark, path_dblp_csv, pub_schema)
#df_dblp = read_csv_with_schema(spark, file_path_dblp, pub_schema)

# Show the DataFrame
df_acm.show()

+--------------------+--------------------+----+-------+--------------------+
|               title|             authors|year|journal|               index|
+--------------------+--------------------+----+-------+--------------------+
|An initial study ...|      Amol Deshpande|2004| SIGMOD|53e9a515b7602d970...|
|Engineering Feder...|Stefan Conrad, Wi...|1999| SIGMOD|53e9b275b7602d970...|
|Information Findi...|Tak W Yan, Hector...|1995| SIGMOD|53e9a5beb7602d970...|
|       Editors Notes|      Jennifer Widom|1995| SIGMOD|53e99800b7602d970...|
|Report on the 5th...|HansJoachim Lenz,...|2003| SIGMOD|53e9a718b7602d970...|
|       Editors Notes|            Ling Liu|2002| SIGMOD|53e99800b7602d970...|
|Report from the N...|Amit P Sheth, Dim...|1996| SIGMOD|53e99e6ab7602d970...|
|TODS Perceptions ...| Richard T Snodgrass|2002| SIGMOD|53e99ae6b7602d970...|
|SQLMED  A Status ...|Jim Melton, JanEi...|2002| SIGMOD|53e9b4d4b7602d970...|
|2003 SIGMOD Innov...| Donald D Chamberlin|2003| SIGMOD|53e9b4a5

In [ ]:
# Filter publications between 1995 and 2004 in VLDB and SIGMOD venues
filtered_df = df.filter(
    (col("year").cast("int").between(1995, 2004)) &
    (col("journal").rlike("(?i)SIGMOD|VLDB"))
)
filtered_df.show()

+--------------------+--------------------+----+--------------------+--------------------+
|               title|             authors|year|             journal|               index|
+--------------------+--------------------+----+--------------------+--------------------+
|The next database...|            Jim Gray|2004|SIGMOD '04 Procee...|5390972920f70186a...|
|The role of crypt...|         Ueli Maurer|2004|SIGMOD '04 Procee...|5390972920f70186a...|
|Tree logical clas...|Stelios Paparizos...|2004|SIGMOD '04 Procee...|5390972920f70186a...|
|Adaptive stream r...|Ankur Jain, Edwar...|2004|SIGMOD '04 Procee...|5390972920f70186a...|
|Holistic UDAFs at...|Graham Cormode, T...|2004|SIGMOD '04 Procee...|5390972920f70186a...|
|Online event-driv...|Huanmei Wu, Betty...|2004|SIGMOD '04 Procee...|5390972920f70186a...|
|Using the structu...|Kristina Lerman, ...|2004|SIGMOD '04 Procee...|5390972920f70186a...|
|FleXPath: flexibl...|Sihem Amer-Yahia,...|2004|SIGMOD '04 Procee...|5390972920f70186a...|

In [ ]:
# Clean the journal column
# Clean the journal column in a single line
cleaned_df = filtered_df.withColumn("journal",
                                    regexp_replace(
                                        regexp_replace("journal", "(?i).*\\bVLDB\\b.*", "VLDB"),
                                        "(?i).*\\bSIGMOD\\b.*", "SIGMOD"))
# Show the cleaned DataFrame
cleaned_df.show(truncate=False)

+-----------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+----+-------+------------------------+
|title                                                                                          |authors                                                                                            |year|journal|index                   |
+-----------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+----+-------+------------------------+
|The next database revolution                                                                   |Jim Gray                                                                                           |2004|SIGMOD |5390972920f70186a0dfac85|
|The role of cryptography in database security          

In [ ]:
# Identify duplicates based on the 'index' column
# duplicate_entries = cleaned_df.groupBy("index").count().where(col("count") > 1)
# duplicate_indices = duplicate_entries.select("index")
# duplicate_indices.show(truncate=False)

+-----+
|index|
+-----+
+-----+



In [ ]:
# #drop them if any
# # df_no_duplicates = cleaned_df.dropDuplicates(subset=["index"])
# # df_no_duplicates.show()
# special_characters_df = cleaned_df.select("title").distinct().select(regexp_extract(col("title"), r'[^a-zA-Z0-9\s]', 0).alias("special_characters"))

# # Show the special characters
# special_characters_df.show(truncate=False)

In [16]:
def remove_null_rows(dataframe):
    return dataframe.dropna()

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, count
from pyspark.sql.window import Window

def find_duplicates(df):
    # Convert titles, authors, and year to lowercase for case-insensitive comparison
    df = df.withColumn("lower_title", lower(col("title"))) \
           .withColumn("lower_authors", lower(col("authors")))

    # Find duplicates using window function
    windowSpec = Window.partitionBy("lower_title", "lower_authors", "year", "index")
    df_with_count = df.withColumn("count", count("*").over(windowSpec))

    # Filter only rows with count > 1, indicating duplicates
    duplicates_df = df_with_count.filter(col("count") > 1).orderBy(col("count").desc()) \
                                 .drop("lower_title", "lower_authors")

    # Remove duplicates from original DataFrame
    df_unique = df_with_count.filter(col("count") == 1).drop("count", "lower_title", "lower_authors")

    return duplicates_df, df_unique

df_acm = remove_null_rows(df_acm)
duplicates, unique_records = find_duplicates(df_acm)

duplicates.show(truncate=False)
#unique_records.show()

+--------------+------------------+----+-------+------------------------+-----+
|title         |authors           |year|journal|index                   |count|
+--------------+------------------+----+-------+------------------------+-----+
|Editors Notes |Ling Liu          |2002|SIGMOD |53e99800b7602d970200b603|4    |
|Editors Notes |Ling Liu          |2002|SIGMOD |53e99800b7602d970200b603|4    |
|Editors Notes |Ling Liu          |2002|SIGMOD |53e99800b7602d970200b603|4    |
|Editors Notes |Ling Liu          |2002|SIGMOD |53e99800b7602d970200b603|4    |
|Chairs Message|M Tamer zsu       |2002|SIGMOD |53e99809b7602d970201f8e7|3    |
|Chairs Message|M Tamer zsu       |2002|SIGMOD |53e99809b7602d970201f8e7|3    |
|Chairs Message|M Tamer zsu       |2002|SIGMOD |53e99809b7602d970201f8e7|3    |
|Editors Notes |Ling Liu          |2004|SIGMOD |53e99800b7602d970200b601|3    |
|Editors Notes |Ling Liu          |2004|SIGMOD |53e99800b7602d970200b601|3    |
|Chairs Message|M Tamer zsu       |2004|

In [18]:
write_to_csv(duplicates, csv_path_duplicates)

In [19]:
write_to_csv(unique_records, "/content/gdrive/MyDrive/Colab Notebooks/Dia_Exercise/unique_dblp.csv")

In [ ]:
pattern = r'[^\w,\s]'
cleaned_df = cleaned_df.withColumn("title", regexp_replace("title", pattern,'')).withColumn("authors", regexp_replace("authors", pattern,''))
cleaned_df.show(truncate=False)

+----------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------+----+-------+------------------------+
|title                                                                                         |authors                                                                                           |year|journal|index                   |
+----------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------+----+-------+------------------------+
|The next database revolution                                                                  |Jim Gray                                                                                          |2004|SIGMOD |5390972920f70186a0dfac85|
|The role of cryptography in database security                  

In [ ]:
cleaned_df.repartition(1).write.format('csv').option("header", "true").option("quote", "\"").option("escape", "\"").save("/content/gdrive/MyDrive/Colab Notebooks/Dia_Exercise/acm_1995_2004_specialCharsTitle.csv", mode='overwrite')

In [ ]:
from pyspark.sql.functions import length, col

# Define the length condition (e.g., filter strings with length greater than 3)
length_condition = filtered_df["year"] == '2004'

# Apply the filter
filtered = filtered_df.filter(length_condition)

# Show the result
filtered.show(truncate=False)

In [ ]:
# @title
import re

# Define a sample title with special characters
title = "Tâ„¢he @Qu.iFAÃ‡AD:Eck, (Br;Ä,Üo-wn) Fo_x Ju\\mps - Oveâ€”r The Laz,a,y Dog isn't  DataMIMEâ„!"

# Remove special characters from the title while retaining commas
cleaned_title = re.sub(r'[^\w,\s]', '', title)

# Print the cleaned title
print("Original Title:", title)
print("Cleaned Title:", cleaned_title)


Original Title: Tâ„¢he @Qu.iFAÃ‡AD:Eck, (Br;Ä,Üo-wn) Fo_x Ju\mps - Oveâ€”r The Laz,a,y Dog isn't  DataMIMEâ„!
Cleaned Title: Tâhe QuiFAÃADEck, BrÄ,Üown Fo_x Jumps  Oveâr The Laz,a,y Dog isnt  DataMIMEâ


In [ ]:
from pyspark.sql.functions import floor
from pyspark.sql.functions import collect_list, explode, udf

num_buckets = 4

# Calculate the range for each bucket
year_range = (2004 - 1995) / num_buckets

# Assign each entry to one or more buckets based on blocking keys (year ranges)
blocked_df = cleaned_df.withColumn("bucket", floor((col("year").cast("int") - 1995) / year_range))

# Group records into blocks based on the buckets
blocked_data = blocked_df.groupBy("bucket").agg(collect_list("index").alias("block_indices"), collect_list("authors"))

# Show blocked data
blocked_data.show(truncate=False)

In [ ]:
cleaned_df.rdd.getNumPartitions()

69

In [ ]:
df_better_partitioned = cleaned_df.repartition(10)
df_better_partitioned.count()

2902

In [ ]:
cleaned_df.count()

2902

In [ ]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

In [ ]:
jaccard_similarity_udf = udf(jaccard_similarity, IntegerType())

NameError: name 'udf' is not defined

In [ ]:
# Apply similarity function to all pairs of entities within each bucket
matched_pairs = blocked_data.select("bucket", explode("block_indices").alias("index1")) \
    .join(df.withColumnRenamed("index", "index1"), col("index1") == col("df.index")) \
    .select("bucket", "index1", split("authors", ", ").alias("authors1")) \
    .join(blocked_data.select("bucket", explode("block_indices").alias("index2")), "bucket") \
    .join(df.withColumnRenamed("index", "index2"), col("index2") == col("df.index2")) \
    .select("index1", "index2", split("authors", ", ").alias("authors2")) \
    .select("index1", "index2", jaccard_similarity_udf("authors1", "authors2").alias("similarity")) \
    .filter("index1 != index2")  # Exclude comparing the same entity

In [ ]:
!pip install python-levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 14.2 MB/s eta 0:00:00


In [ ]:
from Levenshtein import distance, ratio
distance('Levenshtein', 'Lenvinsten')

4

In [21]:
def jaccard_similarity_case_insensitive(str1, str2):
    if str1 is None or str2 is None:
      return 0.0
    set1 = set(str1.lower().split())
    set2 = set(str2.lower().split())

    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))

    similarity = intersection / union if union != 0 else 0
    return similarity

# Example usage:
string1 = "Fernando Berzal, Nicols Marn"
string2 = "Fernando Berzal Galiano, Nicols Marn"
print(jaccard_similarity_case_insensitive(string1, string2))

0.5


In [22]:
# Register the UDF to calculate Jaccard similarity
jaccard_similarity_udf = udf(jaccard_similarity_case_insensitive, DoubleType())

def find_duplicates_for_ground_truth_js(df1, df2, threshold=0.5):
    # Select only the "title" and "authors" columns from df1 and alias them
    df1_subset = df1.select(col("title").alias("title_1"), col("authors").alias("authors_1"))

    # Select only the "title" and "authors" columns from df2 and alias them
    df2_subset = df2.select(col("title").alias("title_2"), col("authors").alias("authors_2"))

    # Join the subsets of df1 and df2 containing "title" and "authors" columns
    joined_df = df1_subset.crossJoin(df2_subset)

    # Calculate the similarity score using Jaccard similarity on "title" and "authors"
    similarity_df = joined_df.withColumn("Title_Similarity", jaccard_similarity_udf(joined_df["title_1"], joined_df["title_2"])) \
                             .withColumn("Authors_Similarity", jaccard_similarity_udf(joined_df["authors_1"], joined_df["authors_2"]))

    # Filter the DataFrame to keep only the pairs with similarity scores greater than or equal to the specified threshold
    duplicates_df = similarity_df.filter((similarity_df["Title_Similarity"] >= threshold) & (similarity_df["Authors_Similarity"] >= threshold))

    # Select and rename the relevant columns
    duplicates_df = duplicates_df.select(duplicates_df["title_1"].alias("Title1"),
                                         duplicates_df["title_2"].alias("Title2"),
                                         duplicates_df["authors_1"].alias("Authors1"),
                                         duplicates_df["authors_2"].alias("Authors2"),
                                         duplicates_df["Title_Similarity"],
                                         duplicates_df["Authors_Similarity"])

    return duplicates_df

In [ ]:
ratio(string1, string2), distance(string1, string2)

(0.875, 8)

In [ ]:
jaccard_similarity('Tucakov V.', 'tucano v.')

AttributeError: 'str' object has no attribute 'intersection'

In [ ]:
ratio("Edwin M. Knorr, Raymond T. Ng, V. Tucakov", "Edwin M. Knorr, Raymond T. Ng, Vladimir Tucakov")

In [ ]:
def generate_id(title, authors, venue, year):
    # Split the authors' names by commas
    author_list = authors.split(', ')

    # Extract the last name of the first author
    first_author_last_name = author_list[0].split()[-1]

    # Take the first letter of the last name for other authors
    other_authors_initials = ''.join([author.split()[-1][0].capitalize() for author in author_list[1:]])

    # Extract the last two digits of the year
    year_last_two_digits = str(year)[-2:]

    # Concatenate the components to form the id
    id_string = f"{first_author_last_name}{other_authors_initials}{year_last_two_digits}"

    return id_string

# Example usage
title = "Phrase Matching in XML"
authors = "Sihem Amer-Yahia, Mary F. Fernandez, Yu Xu, Divesh Srivastava"
venue = "VLDB"
year = 2003

id = generate_id(title, authors, venue, year)
print(id)  # Output: "conf/vldb/AmerFYDS03

Amer-YahiaFXS03


In [ ]:
def generate_key(title, authors):
    # Shorten the title by taking the first word and the first letter of each subsequent word
    title_parts = title.split()
    shortened_title = '-'.join([word[:1] for word in title_parts])

    # Extract initials of authors
    author_initials = ''.join([name[0] for name in authors.split()])

    # Concatenate the shortened title and author initials to form the key
    key = f"{shortened_title}-{author_initials}"
    return key

# Example usage:
title_1 = "Onedimensional and multidimensional substring selectivity estimation"
authors_1 = "H V Jagadish, Olga Kapitskaia, Raymond T Ng, Divesh Srivastava"
key_1 = generate_key(title_1, authors_1)
print("Key for Record 1:", key_1)

title_2 = "MultiDimensional Substring Selectivity Estimation"
authors_2 = "H V Jagadish, Olga Kapitskaia, Raymond T Ng, Divesh Srivastava"
key_2 = generate_key(title_2, authors_2)
print("Key for Record 2:", key_2)


Key for Record 1: O-a-m-s-s-e-HVJOKRTNDS
Key for Record 2: M-S-S-E-HVJOKRTNDS


In [ ]:
def tokenize(text):
    """
    Tokenizes the input text by splitting it into tokens.
    """
    return text.split()

def create_blocks(records, token_type):
    """
    Creates blocks based on common tokens shared between titles or authors' names.

    Args:
    - records: List of records, where each record is a tuple containing (title, authors, venue, year).
    - token_type: Type of tokens to use for blocking ('title' or 'author').

    Returns:
    - Dictionary where keys are tokens and values are lists of records containing that token.
    """
    blocks = {}
    for record in records:
        title, authors, venue, year = record
        tokens = tokenize(title) if token_type == 'title' else tokenize(authors)
        for token in tokens:
            if token not in blocks:
                blocks[token] = []
            blocks[token].append(record)
    return blocks

# Example dataset
dataset = [
    ("Phrase Matching in XML", "Sihem Amer-Yahia, Mary F. Fernandez, Yu Xu, Divesh Srivastava", "VLDB", 2003),
    # Add other records from the dataset here
]

# Create blocks based on author tokens
author_blocks = create_blocks(dataset, 'author')

# Print the blocks
for token, records in author_blocks.items():
    print(f"Block for author token '{token}':")
    for record in records:
        print(record)
    print()


Block for author token 'Sihem':
('Phrase Matching in XML', 'Sihem Amer-Yahia, Mary F. Fernandez, Yu Xu, Divesh Srivastava', 'VLDB', 2003)

Block for author token 'Amer-Yahia,':
('Phrase Matching in XML', 'Sihem Amer-Yahia, Mary F. Fernandez, Yu Xu, Divesh Srivastava', 'VLDB', 2003)

Block for author token 'Mary':
('Phrase Matching in XML', 'Sihem Amer-Yahia, Mary F. Fernandez, Yu Xu, Divesh Srivastava', 'VLDB', 2003)

Block for author token 'F.':
('Phrase Matching in XML', 'Sihem Amer-Yahia, Mary F. Fernandez, Yu Xu, Divesh Srivastava', 'VLDB', 2003)

Block for author token 'Fernandez,':
('Phrase Matching in XML', 'Sihem Amer-Yahia, Mary F. Fernandez, Yu Xu, Divesh Srivastava', 'VLDB', 2003)

Block for author token 'Yu':
('Phrase Matching in XML', 'Sihem Amer-Yahia, Mary F. Fernandez, Yu Xu, Divesh Srivastava', 'VLDB', 2003)

Block for author token 'Xu,':
('Phrase Matching in XML', 'Sihem Amer-Yahia, Mary F. Fernandez, Yu Xu, Divesh Srivastava', 'VLDB', 2003)

Block for author token 'D

In [ ]:
# Create two small DataFrames with the same column header
df1 = spark.createDataFrame([(1, 'A'), (2, 'B')], ['id', 'value'])
df2 = spark.createDataFrame([(3, 'X'), (4, 'Y')], ['id', 'value'])

# Perform cross join
cross_joined_df = df1.crossJoin(df2)

# Show the result
cross_joined_df.show()

+---+-----+---+-----+
| id|value| id|value|
+---+-----+---+-----+
|  1|    A|  3|    X|
|  1|    A|  4|    Y|
|  2|    B|  3|    X|
|  2|    B|  4|    Y|
+---+-----+---+-----+



In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

def levenshtein_ratio(s1, s2):
    return ratio(s1, s2)

# Register the UDF to calculate Levenshtein ratio
levenshtein_ratio_udf = udf(levenshtein_ratio, DoubleType())

def find_duplicates(df1, df2, threshold=0.5):
    # Select only the "title" and "authors" columns from df1 and alias them
    df1_subset = df1.select(col("title").alias("title_1"), col("authors").alias("authors_1"))

    # Select only the "title" and "authors" columns from df2 and alias them
    df2_subset = df2.select(col("title").alias("title_2"), col("authors").alias("authors_2"))

    # Join the subsets of df1 and df2 containing "title" and "authors" columns
    joined_df = df1_subset.crossJoin(df2_subset)

    # Calculate the similarity score using Levenshtein ratio on "title" and "authors"
    similarity_df = joined_df.withColumn("Title_Similarity", levenshtein_ratio_udf(joined_df["title_1"], joined_df["title_2"])) \
                             .withColumn("Authors_Similarity", levenshtein_ratio_udf(joined_df["authors_1"], joined_df["authors_2"]))

    # Filter the DataFrame to keep only the pairs with similarity scores greater than or equal to the specified threshold
    duplicates_df = similarity_df.filter((similarity_df["Title_Similarity"] >= threshold) & (similarity_df["Authors_Similarity"] >= threshold))

    # Select and rename the relevant columns
    duplicates_df = duplicates_df.select(duplicates_df["title_1"].alias("Title1"),
                                         duplicates_df["title_2"].alias("Title2"),
                                         duplicates_df["authors_1"].alias("Authors1"),
                                         duplicates_df["authors_2"].alias("Authors2"),
                                         duplicates_df["Title_Similarity"],
                                         duplicates_df["Authors_Similarity"])

    return duplicates_df


In [ ]:

data = rdd.map(lambda x: tuple(
    field[2:].split('\n') if field.startswith('#%') else
    field[6:] if field.startswith('#index') else
    field[2:].split(',') if field.startswith("#@") else
    field[2:]
    for field in filter(None, x[1].split('\n'))
))

data = data.map(lambda x: {"title": x[0], "authors": x[1], "year": x[2], "journal": x[3], "index": x[4], "refid": [item for sublist in x[5:] for item in sublist]})

# Define the schema
schema = StructType([
    StructField("title", StringType(), True),
    StructField("authors", ArrayType(StringType(), True)),
    StructField("year", StringType(), True),
    StructField("journal", StringType(), True),
    StructField("index", StringType(), True),
    StructField("refid", ArrayType(StringType(), True)),
])

# # Apply the schema to the RDD 'a'
df = spark.createDataFrame(data, schema=schema)

#df.write.format("noop").mode("overwrite").save()

# # Show the DataFrame
#df.show(truncate=False)

In [ ]:
data = rdd.filter(lambda x: not x[1].startswith('#%')).map(lambda x: tuple(
    field
    for field in filter(None, x[1].split('\n')) if not field.startswith('#%')
))

In [ ]:
data = rdd.filter(lambda x: not x[1].startswith('#%')).map(lambda x: tuple(
    (
        next((field[2:] for field in x[1].splitlines() if field.startswith('#*')), None),  # Paper Title
        next((field[2:] for field in x[1].splitlines() if field.startswith('#@')), None),  # Authors
        next((field[2:] for field in x[1].splitlines() if field.startswith('#t')), None),  # Year
        next((field[2:] for field in x[1].splitlines() if field.startswith('#c')), None),  # Publication Venue
        next((field[6:] for field in x[1].splitlines() if field.startswith('#index')), None)  # Index ID
    )
))

for el in data.take(5):
    print(el)


In [ ]:
example_string = '''#*Report on the First "XQuery Implementation, Experience, and Perspectives" Workshop (XIME-P).
#@Ioana Manolescu, Yannis Papakonstantinou
#t2004
#cSIGMOD Record
#index53e9b152b7602d9703bd3739'''

for el in example_string.splitlines():
    print(el)


In [ ]:
for el in rdd.take(5):
    print(el)

In [ ]:


# for el in string.split('\n'):
#   if el.startswith("#*") or el.rfind(" \" "):
#     print(el[2:])
#   elif el.startswith("#@"):
#     print(el[2:])
#   elif el.startswith("#t"):
#     print(el[2:])
#   elif el.startswith("#c"):
#     print(el[2:])
#   elif el.startswith("#index"):
#     print(el[6:])
#   else:
#     print(None)

In [ ]:
data = rdd.filter(lambda x: not x[1].startswith('#%')).map(lambda x: tuple(
    (
        next((field[2:] for field in str(x[1]).split('\n') if field.startswith('#*')), None),  # Paper Title
        next((field[2:] for field in x[1].split('\n') if field.startswith('#@')), None),  # Authors
        next((field[2:] for field in x[1].split('\n') if field.startswith('#t')), None),  # Year
        next((field[2:] for field in x[1].split('\n') if field.startswith('#c')), None),  # Publication Venue
        next((field[6:] for field in x[1].split('\n') if field.startswith('#index')), None)  # Index ID
    )
))

for el in data.take(5):
    print(el)

In [ ]:
for el in data.take(5):
  print(el)

In [ ]:
data_task1 = rdd.filter(lambda x: not x[1].startswith('#%')).map(lambda x: tuple(
    extract_value(field, '#*',2) if field.startswith('#*') else  # title
    extract_value(field, '#@', 2) if field.startswith('#@') else  # authors
    extract_value(field, '#t', 2) if field.startswith('#t') else  # year
    extract_value(field, '#c', 2) if field.startswith('#c') else  # venue
    extract_value(field, '#index', 6) if field.startswith('#index') else None  # id
    for field in filter(None, x[1].split('\n'))
))

def extract_value(field, marker, idx):
    return field[idx:] if field.startswith(marker) else None


In [ ]:
for el in data_task1.take(5):
  print(el)

In [ ]:
for el in rdd.take(5):
  print(el)

In [ ]:
data_task1 = rdd.filter(lambda x: not x[1].startswith('#%')).map(lambda x: tuple(
    field[2:] if field.startswith('#*') else # title
    field[2:] if field.startswith('#@') else  # authors
    field[2:] if field.startswith('#t') else  # year
    field[2:] if field.startswith('#c') else  # venue
    field[6:] if field.startswith('#index') else # id
    None
    for field in filter(None, x[1].split('\n')) if not field.startswith('#%')
))
for el in data_task1.take(9):
  print(el)

In [ ]:
contains_none = data_task1.filter(lambda x: None in x).count() > 0
if contains_none:
    print("data_task1 contains at least one None value.")
else:
    print("data_task1 does not contain any None values.")


In [ ]:
# Define the schema
schema1 = StructType([
    StructField("title", StringType(), True),
    StructField("authors", StringType(), True),
    StructField("year", StringType(), True),
    StructField("journal", StringType(), True),
    StructField("index", StringType(), True),
])

# # Apply the schema to the RDD 'a'
df1 = spark.createDataFrame(data, schema=schema1)

#df.write.format("noop").mode("overwrite").save()

# # Show the DataFrame
df1.show(truncate=False)

In [ ]:
# # Find the maximum length of any tuple
# max_length = data.map(len).max()

# # Pad the tuples with null values to match the maximum length
# padded_data = data.map(lambda x: x + (None,) * (max_length - len(x)))

# # Create a schema with fields corresponding to the maximum length
# schema2 = StructType([
#     StructField(f"field_{i+1}", StringType(), True)
#     for i in range(max_length)
# ])

da = data.map(lambda x: {"title": x[0], "authors": x[1], "year": x[2], "journal": x[3], "index": x[4]})

# Create DataFrame with inferred schema and padded data
df2 = spark.createDataFrame(da, schema=schema1)
df2.show(truncate=False)

In [ ]:
df2.printSchema()

In [ ]:
# data_task1 = rdd.map(lambda x: tuple(
#     field[6:] if field.startswith('#index') else
#     field[2:]
#     for field in filter(None, x[1].split('\n'))
# ))

# # Debugging print statement
# print("Data from data_task1:")
# print(data_task1.take(5))

# # Filter data based on publication year and venue
# data_1 = data_task1.filter(lambda x: x[2].isdigit() and 1995 <= int(x[2]) <= 2004 and any(venue.lower() in x[3].lower() for venue in ["SIGMOD", "VLDB"]))

# Debugging print statement
#print("Data after filtering:")
#print(data_1.take(5))

#data_1 = data_task1.map(lambda x: {"title": x[0], "authors": x[1], "year": x[2], "journal": x[3], "index": x[4]})

# Define the schema
schema1 = StructType([
    StructField("title", StringType(), True),
    StructField("authors", StringType(), True),
    StructField("year", StringType(), True),
    StructField("journal", StringType(), True),
    StructField("index", StringType(), True),
])

# # Apply the schema to the RDD 'a'
df1 = spark.createDataFrame(data_task1, schema=schema1)

#df.write.format("noop").mode("overwrite").save()

# # Show the DataFrame
df1.show(truncate=False)

In [ ]:
# Filter publications between 1995 and 2004 in VLDB and SIGMOD venues
filtered_df = df2.filter(
    (col("year").cast("int").between(1995, 2004)) &
    (col("journal").rlike("(?i)SIGMOD|VLDB"))
)

# # Concatenate authors array into a single string separated by a comma
# filtered_df = filtered_df.withColumn("authors_str", concat_ws(", ", col("authors")))

# # Select the required columns
# selected_columns = ["index", "title", "authors_str", "journal", "year"]
# filtered_df = filtered_df.select(selected_columns)

filtered_df.show()

In [ ]:

from pyspark.sql.functions import length, col

# Define the length condition (e.g., filter strings with length greater than 5)
length_condition = length(filtered_df["year"]) > 4

# Apply the filter
filtered = filtered_df.filter(length_condition)

# Show the result
filtered.show()

In [ ]:
from pyspark.sql.functions import length, col
# Define the length condition using col("column_name")
length_condition = length(col("year")) > 5

# Apply the filter
filtered = filtered_df.filter(length_condition)

# Show the result
filtered.show()

In [ ]:
# Write the result to CSV files
filtered_df.write.csv("/content/gdrive/MyDrive/DBLP_1995_2004_test.csv", header=True, mode="overwrite")

In [ ]:
filtered_df.repartition(1).write.csv("/content/gdrive/MyDrive/DBLP_1995_2004_test3.csv", header=True, mode="overwrite")


In [ ]:

# Select the required columns
selected_columns = ["index", "title", "authors", "journal", "year"]
filtered_df = filtered_df.select(selected_columns)



# # Write the result to CSV files
filtered_df.write.csv("/content/gdrive/MyDrive/DBLP_1995_2004.csv", header=True, mode="overwrite")

# # Additional filtering for ACM venue
# acm_filtered_df = df.filter(
#     (col("year").cast("int").between(1995, 2004)) &
#     (col("journal").rlike("(?i)ACM"))
# )

# acm_filtered_df = acm_filtered_df.select(selected_columns)
# # acm_filtered_df.write.csv("ACM_1995_2004.csv", header=True, mode="overwrite")

# Stop the Spark session
#spark.stop()
